In [1]:
import pandas as pd
import numpy as np
from tools import ItemEncoder, get_rec_als, split_data
from prepare_data import make_data
from scipy import sparse as sp
from tqdm.notebook import tqdm
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from implicit.evaluation import train_test_split, ndcg_at_k, precision_at_k, AUC_at_k
from catboost import CatBoostRanker, Pool, MetricVisualizer
from sklearn.model_selection import train_test_split
from tools import compute_metrics
from copy import deepcopy

In [2]:
metrics = pd.read_csv('files/metrics.csv',index_col=[0])

In [3]:
metrics

,name_model,Precision@20,Recall@20,MAP@20,MRR
0,implicit_als,0.006300,0.033383,0.009916,0.029859
1,implicit_als_ctb_class,0.008508,0.043602,0.013360,0.029859
0,implicit_als_ctb_rank,0.002676,0.012161,0.003872,0.029859
0,implicit_als_rank2split,0.003089,0.014593,0.004338,0.029859


In [ ]:
train = pd.read_parquet('files/vk_data/train.parquet.gzip')
test = pd.read_parquet('files/vk_data/fresh_candidates.parquet.gzip')

In [ ]:
user_features = pd.read_parquet('files/user_features.parquet.gzip').reset_index()
item_features = pd.read_parquet('files/item_features.parquet.gzip')

In [ ]:
#ALS candidates
coder = ItemEncoder(train)
user_items_interaction = coder.make_csr_data(train)
user_items_weights = coder.make_csr_data(train, weights='timespent')
bm25_int = bm25_weight(user_items_interaction, K1=80, B=0.8).tocsr()
bm25_weights = bm25_weight(user_items_weights, K1=80, B=0.8).tocsr()

In [ ]:
model_als = AlternatingLeastSquares(factors=200, iterations=2, num_threads=-1, random_state=42)
model_als.fit(bm25_weights)
users, rec = get_rec_als(model_als, local_train['user_id'].unique(), bm25_weights, coder)
candidates_als = pd.DataFrame({'user_id':users, 'item_id': rec})
candidates_als.head()

In [ ]:
df_predict_als = candidates_als.explode('item_id').astype('int32')
df_predict_als['rank'] = (df_predict_als.groupby('user_id').cumcount() + 1).astype('int32')
df_predict_als.head()

In [ ]:
model = CatBoostRanker()
#model.load_model('files/ctb_rank')
#model.load_model('files/ctb_rank_2split_data')
#model.load_model('files/ctb_rank_2split_data')
model.load_model('files/ctb_rank_2split_data_quant')

In [ ]:
als_ctb_data = df_predict_als.merge(item_features[item_col], 
                                on=['item_id'], 
                                how='left'
                               ).merge(user_features, on=['user_id'], how='left').sort_values(by=['user_id'])
als_ctb_data = als_ctb_data.drop(columns=['user_id', 'item_id'])
#als_ctb_data.to_parquet('files/als_ctb_data_full_test.parquet.gzip')
als_ctb_data .head()

In [ ]:
full_test_p = Pool(
    data=als_ctb_data,
    group_id=qied_full_test,
    cat_features=['source_id'],
    #embedding_features=['embeddings']
)

In [ ]:
df_predict = predict_rank(model, full_test_p, df_predict_als)